In [1]:
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import albumentations as A
from albumentations.core.composition import OneOf
import glob
import os
import numpy as np

In [15]:
df = pd.read_csv('/workspace/#Ub370#Uc774#Ud130/test/label/test.csv')
df.head()

,filename,maxX,maxY,minX,minY,faceExp_uploader,path
0,q347e9fe539cb67cc112ec595fa351e0ae0ab1e57027fb...,2426.120954,1755.055182,1531.498779,578.357093,분노,/workspace/#Ub370#Uc774#Ud130/test/image/anger/
1,n8k53439edaa3fd087cd64e0cdffb51b3ed78af9476c79...,1950.096587,1652.864682,1100.336132,512.750743,분노,/workspace/#Ub370#Uc774#Ud130/test/image/anger/
2,zm6t9255f092886de2c9bc8819c55cee98d28c1a36500e...,1625.204932,1786.189538,814.467386,703.615438,분노,/workspace/#Ub370#Uc774#Ud130/test/image/anger/
3,xdq10b215a7aabf82c30cb477cdd4823e8c1baa4838201...,2921.362497,1391.565440,2194.335797,515.530543,분노,/workspace/#Ub370#Uc774#Ud130/test/image/anger/
4,o312ddd59e8f15955220465e825529352f6d1ed1b50513...,2034.405881,1511.470155,1243.836628,430.100521,분노,/workspace/#Ub370#Uc774#Ud130/test/image/anger/


In [16]:
df['faceExp_uploader'].value_counts()

faceExp_uploader
분노    300
기쁨    300
당황    300
슬픔    300
Name: count, dtype: int64

In [17]:

# .npz 파일 로드
npz_train_anger = np.load('/workspace/#Ub370#Uc774#Ud130/test/segmentation/test_anger.npz')
npz_train_happy = np.load('/workspace/#Ub370#Uc774#Ud130/test/segmentation/test_happy.npz')
npz_train_panic = np.load('/workspace/#Ub370#Uc774#Ud130/test/segmentation/test_panic.npz')
npz_train_sadness = np.load('/workspace/#Ub370#Uc774#Ud130/test/segmentation/test_sadness.npz')


In [19]:
processed_data = []

for index, row in df.iterrows():
    # if index == 5:
    #     break
    if index % 100 == 0:
        print(f'{index} / {len(df)}')

    npz_train = npz_train_anger
    if row['faceExp_uploader'] == '기쁨':
        npz_train = npz_train_happy
    elif row['faceExp_uploader'] == '당황':
        npz_train = npz_train_panic
    elif row['faceExp_uploader'] == '슬픔':
        npz_train = npz_train_sadness

    data = npz_train[row['filename']]  # Segmentation 데이터 로드

    image = cv2.imread(row['path'] + row['filename'])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 원본 이미지의 크기 (width, height)
    original_height, original_width = image.shape[:2]

    # 새로운 이미지 생성 (검정색 초기화)
    new_img = image.copy()
    new_img[:, :] = [0, 0, 0]  # 검정색으로 초기화
    mask = data != 3  # 얼굴 영역 추출
    new_img[mask] = image[mask]  # 얼굴만 원본 이미지에서 가져와 새로운 이미지에 덮어씌움

    image_path_anger = '/workspace/Data_Augumentation/segmentation_no_face/image/test/anger/'
    image_path_happy = '/workspace/Data_Augumentation/segmentation_no_face/image/test/happy/'
    image_path_panic = '/workspace/Data_Augumentation/segmentation_no_face/image/test/panic/'
    image_path_sadness = '/workspace/Data_Augumentation/segmentation_no_face/image/test/sadness/'

    # 얼굴만 남긴 이미지를 640x640으로 리사이즈 후 저장
    output_file_path = os.path.join(image_path_anger, f"no_face_{row['filename']}")
    path = image_path_anger
    if row['faceExp_uploader'] == '기쁨':
        output_file_path = os.path.join(image_path_happy, f"no_face_{row['filename']}")
        path = image_path_happy
    elif row['faceExp_uploader'] == '당황':
        output_file_path = os.path.join(image_path_panic, f"no_face_{row['filename']}")
        path = image_path_panic
    elif row['faceExp_uploader'] == '슬픔':
        output_file_path = os.path.join(image_path_sadness, f"no_face_{row['filename']}")
        path = image_path_sadness

    new_img_bgr = cv2.cvtColor(new_img, cv2.COLOR_RGB2BGR)  # OpenCV는 BGR 포맷 사용
    resized_img = cv2.resize(new_img_bgr, (640, 640))  # 리사이즈
    cv2.imwrite(output_file_path, resized_img)  # 이미지 저장

    # 바운딩 박스 크기 비율 계산
    width_ratio = 640 / original_width
    height_ratio = 640 / original_height

    new_x_min = int(row['minX'] * width_ratio)
    new_y_min = int(row['minY'] * height_ratio)
    new_x_max = int(row['maxX'] * width_ratio)
    new_y_max = int(row['maxY'] * height_ratio)

    # 변경된 데이터 저장
    processed_data.append({
        'filename': f"no_face_{row['filename']}",
        'maxX': new_x_max,
        'maxY': new_y_max,
        'minX': new_x_min,
        'minY': new_y_min,
        'faceExp_uploader': row['faceExp_uploader'],
        'path': path
    })

output_csv_path = '/workspace/Data_Augumentation/segmentation_no_face/label/test/no_face_test.csv'

# 처리된 데이터 저장
if processed_data:
    processed_df = pd.DataFrame(processed_data)
    os.makedirs(os.path.dirname(output_csv_path), exist_ok=True)
    processed_df.to_csv(output_csv_path, index=False)
    print(f"Processed CSV saved to: {output_csv_path}")
else:
    print("No data processed.")
    


0 / 1200


Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG


100 / 1200


Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG


200 / 1200


...

1000 / 1200


Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG


1100 / 1200


Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG


Processed CSV saved to: /workspace/Data_Augumentation/segmentation_no_face/label/test/no_face_test.csv
